In [1]:
import numpy as np
import re
from nltk.corpus import stopwords
import csv


# The `stop` is defined as earlier in this chapter
# Added it here for convenience, so that this section
# can be run as standalone without executing prior code
# in the directory
stop = stopwords.words('german')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    #tokenized = [w for w in text.split()]
    return tokenized
         

In [2]:
from nltk.stem import SnowballStemmer 

def stem_sentence(sentence):
    words = tokenizer(sentence)

    # print(" ".join(SnowballStemmer.languages))

    #print("\nSnowballStemmer (Porter vs. German):")
    sPorter = SnowballStemmer("porter")
    sGerman = SnowballStemmer("german")
    
    #for w in words: 
        #print(" => ", w, ":", sPorter.stem(w), "|", sGerman.stem(w))
    

In [3]:
def stream_docs(path):
    #with open(path, 'rb') as csvfile:
   #     csv_reader = csv.reader(csvfile, delimiter=';', quotechar='\'')
   #     for line in csv_reader:
    #        text, label = line[:-3], int(line[-2])
    #        yield text, label     
    
    with open(path, 'r', encoding='utf-8') as csv:
        #next(csv)  # skip header
        for line in csv:
            #print(line)
            text, label = line[:-3], int(line[-2])
            text = stem_sentence(text)
            yield text, label   

In [4]:
next(stream_docs(path='trainingsdata/training_gesamt_extended.csv'))


SnowballStemmer (Porter vs. German):


(None, 1)

In [5]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [6]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [7]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

if Version(sklearn_version) < '0.18':
    clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
else:
    clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

doc_stream = stream_docs(path='trainingsdata/training_gesamt_extended.csv')

In [8]:
#Only needed for progressbar
import pyprind
pbar = pyprind.ProgBar(35)

classes = np.array([0, 1])
for _ in range(35):
    X_train, y_train = get_minibatch(doc_stream, size=200)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()


SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballStemmer (Porter vs. German):

SnowballSte

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=900)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [ ]:
doc_stream = stream_docs(path='trainingsdata/german_validation.csv')

X_val, y_val = get_minibatch(doc_stream, size=7)
X_val = vect.transform(X_val)
print('Accuracy: %.3f' % clf.score(X_val, y_val))

In [ ]:
predictions = clf.predict(X_val)
print(predictions)